In [37]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from datetime import datetime as dt

# normal times


In [44]:
df_arima = pd.read_csv('./../../error_dfs/error_2018_2019_ARIMA.csv', converters={'ags2': str, 'ags5': str} )
df_nn = pd.read_csv('./../../error_dfs/error_2018_2019_nn.csv', converters={'ags2': str, 'ags5': str} )
df_var = pd.read_csv('./../../error_dfs/error_2018_2019_VAR.csv', converters={'ags2': str, 'ags5': str} )

df_arima = strip_day(df_arima)
df_nn = strip_day(df_nn)
df_var = strip_day(df_var)

df_arima = fix_ags5(df_arima)
df_nn = fix_ags5(df_nn)
df_var = fix_ags5(df_var)

df = pd.merge(df_arima, df_nn, on = ['ags5', 'date','ground_truth'])
df = pd.merge(df, df_var, on = ['ags5', 'date','ground_truth'])
df

,ags5,date,ground_truth,pred_x,error_x,pred_y,error_y,pred
0,01001,2018-01,9.3,9.028416,0.271584,8.666405,0.633595,9.100280
1,01001,2018-02,9.2,9.031855,0.168145,8.807995,0.392005,9.081553
2,01001,2018-03,9.1,8.909250,0.190750,8.730574,0.369426,8.925309
3,01001,2018-04,8.8,8.975852,0.175852,8.242456,0.557545,8.987270
4,01001,2018-05,8.3,8.727940,0.427940,8.107006,0.192994,8.770163
...,...,...,...,...,...,...,...,...
9619,16077,2019-08,7.0,7.006511,0.006511,7.114399,0.114399,6.928729
9620,16077,2019-09,6.5,6.842809,0.342809,7.089566,0.589566,6.595042
9621,16077,2019-10,6.5,6.451817,0.048183,6.721840,0.221840,6.526319
9622,16077,2019-11,6.3,6.456085,0.156085,6.756373,0.456373,6.483260


In [43]:
def strip_day(df): 
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.strftime('%Y-%m')
    return df

In [14]:
def fix_ags5(df): 
    df['ags5'] = np.where(df.ags5.str.len() ==4, '0' + df['ags5'], df['ags5'])
    return df

In [48]:
predicted = (df['pred'] + df['pred_x'] + df['pred_y'])/3 

mean_squared_error(df['ground_truth'], predicted)

0.02670377604830163

## Covid times

In [80]:
df_arima = pd.read_csv('./../../error_dfs/error_2019_may2021_sarima.csv', converters={'ags2': str, 'ags5': str} )
df_nn = pd.read_csv('./../../error_dfs/error_2019_may2021_nn.csv', converters={'ags2': str, 'ags5': str} )
df_var = pd.read_csv('./../../error_dfs/error_2019_may2021_VAR.csv', converters={'ags2': str, 'ags5': str} )


df_arima = strip_day(df_arima)
df_nn = strip_day(df_nn)
df_var = strip_day(df_var)

df_arima = fix_ags5(df_arima)
df_nn = fix_ags5(df_nn)
df_var = fix_ags5(df_var)

df_var = df_var[df_var['date'] > '2018-12']

df = pd.merge(df_arima, df_nn, on = ['ags5', 'date','ground_truth'])
df = pd.merge(df, df_var, on = ['ags5', 'date','ground_truth'])
df

,ags5,date,ground_truth,pred_x,error_x,pred_y,error_y,pred
0,01001,2019-01,8.2,7.801409,0.398591,7.778496,0.421504,8.038843
1,01001,2019-02,7.8,7.746762,0.053238,7.848810,0.048810,8.022765
2,01001,2019-03,7.8,7.625305,0.174695,7.825788,0.025788,7.754631
3,01001,2019-04,7.6,7.634096,0.034096,7.566484,0.033517,7.502805
4,01001,2019-05,7.7,7.362508,0.337492,7.368903,0.331097,6.906987
...,...,...,...,...,...,...,...,...
11624,16077,2021-01,7.3,7.177730,0.122270,6.728317,0.571683,7.479328
11625,16077,2021-02,7.3,7.148730,0.151270,6.785843,0.514157,7.435923
11626,16077,2021-03,7.2,6.925377,0.274623,6.749915,0.450085,7.046545
11627,16077,2021-04,6.9,7.046128,0.146128,6.886751,0.013249,6.780956


In [81]:
predicted = (df['pred'] + df['pred_x'] + df['pred_y'])/3 

mean_squared_error(df['ground_truth'], predicted)

0.17370070936065626

In [82]:
mean_squared_error(df_arima['ground_truth'], df_arima['pred'])

0.18396048394403136

In [77]:
df_var

,date,ground_truth,pred,ags5
1,2019-01,8.2,8.038843,01001
2,2019-02,7.8,8.022765,01001
3,2019-03,7.8,7.754631,01001
4,2019-04,7.6,7.502805,01001
5,2019-05,7.7,6.906987,01001
...,...,...,...,...
12025,2021-01,7.3,7.479328,16077
12026,2021-02,7.3,7.435923,16077
12027,2021-03,7.2,7.046545,16077
12028,2021-04,6.9,6.780956,16077
